In [1]:
import os, getpass
from pandas import read_csv

In [2]:
import pandas as pd
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai import Credentials

In [ ]:
credentials = {
    "url" : "https://eu-de.ml.cloud.ibm.com" ,
    "apikey" : getpass.getpass("Enter your API key: ")
}

Enter your API key:  ········


In [4]:
try:
    project_id=os.environ["PROJECT_ID"]

except KeyError:
    project_id = input("please enter your project_id:")

In [5]:
project_id

'0687afb3-030b-4c62-b2a7-a900db677b28'

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='gI-RvDaGDHqk0IRGXPPiAXcgcghv1SVVK3C63f-zNEyx',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'project1-donotdelete-pr-bmlokidkz3xa5x'
object_key = 'emails.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_1 = pd.read_csv(body)
df_1.head(10)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...
7,allen-p/_sent_mail/1004.,Message-ID: <17189699.1075863688308.JavaMail.e...
8,allen-p/_sent_mail/101.,Message-ID: <20641191.1075855687472.JavaMail.e...
9,allen-p/_sent_mail/102.,Message-ID: <30795301.1075855687494.JavaMail.e...


In [10]:
model = Model(
    model_id="ibm/granite-13b-instruct-v2",  # You can also try "granite-13b-instruct-v1"
    credentials=credentials,
    project_id= project_id,
    params={
        GenParams.MAX_NEW_TOKENS: 200,
        GenParams.TEMPERATURE: 0.5,
        GenParams.DECODING_METHOD: "greedy"
    }
)

/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/model.py:105: DeprecationWarning: The `Model` class is deprecated and will be removed in a future release. Please use the `ModelInference` class instead. To update your imports, use: `from ibm_watsonx_ai.foundation_models import ModelInference`.
  warn(model_class_deprecated_warning, category=DeprecationWarning)


In [18]:
def extract_tasks(email_text):
    prompt = f"""
You are a helpful assistant. Extract clear and actionable tasks from the following email and list them as bullet points.

Email:
\"\"\"{email_text}\"\"\"

Tasks:
"""
    response = model.generate_text(prompt=prompt)
    return response.strip()

In [20]:
df_sample = df_1.head(5)

In [21]:
print(df_sample.columns)

Index(['file', 'message'], dtype='object')


In [23]:
for idx, row in df_sample.iterrows():
    print(f"Email {idx+1}:\n{row['message']}")
    tasks = extract_tasks(row['message'])
    print(f"Extracted Tasks:\n{tasks}\n")

Email 1:
Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 
Extracted Tasks:
- Read the attached file.
- Extract the forecast.

Email 2:
Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>
Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'